In [ ]:
import numpy as np
import pandas as pd
from time import perf_counter
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [ ]:
def random_search(distance_matrix, current_node_index=None):
    to_visit = set(range(len(distance_matrix)))
    
    if current_node_index is None:
        current_node_index = np.random.choice(list(to_visit))
    
    current_node = current_node_index
    solution = [current_node]
    total_cost = distance_matrix[current_node, current_node]
    to_visit.remove(current_node_index)
    
    while len(solution) != len(distance_matrix) // 2:
        next_node_index = np.random.choice(list(to_visit))
        next_node = next_node_index
        total_cost += distance_matrix[current_node, next_node]
        solution.append(next_node)
        to_visit.remove(next_node_index)
        current_node = next_node
    
    total_cost += distance_matrix[solution[-1], solution[0]]
    
    return solution, total_cost


In [ ]:
def calculate_euclidean_distance(node1, node2):
    return np.round(np.sqrt((node1['x'] - node2['x'])**2 + (node1['y'] - node2['y'])**2))


def get_distance_matrix(df, costs=False):
    num_nodes = df.shape[0]
    distance_matrix = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        node1 = df.iloc[i]
        for j in range(i, num_nodes):
            node2 = df.iloc[j]
            distance = calculate_euclidean_distance(node1, node2)
            if costs:
                distance_matrix[i,j] = distance + node2['cost']
                distance_matrix[j,i] = distance + node1['cost']
            else:
                distance_matrix[i,j] = distance
                distance_matrix[j,i] = distance
    return distance_matrix

In [ ]:
def get_node_moves(solution, flag):
    n = len(solution)
    moves = list()
    if flag == 'intra':
        for i in range(len(solution) - 1):
            for j in range(i+1, len(solution)):
                moves.append((solution[i], solution[j]))
    elif flag == 'inter':
        outer_nodes = list(set(range(int(n*2))) - set(solution))
        for i in range(len(solution)):
            for j in range(len(outer_nodes)):
                moves.append((solution[i], outer_nodes[j]))   
    return moves

In [ ]:
def get_edge_moves(solution):
    moves = list()
    for i in range(len(solution) - 1):
        for j in range(i + 2, len(solution) - 1):
            edge1 = (solution[i], solution[i + 1])
            edge2 = (solution[j], solution[j + 1])
            moves.append((edge1, edge2))
    return moves

In [ ]:
def compute_node_swap_delta(solution, swap, distance_matrix, costs, flag):
    new_solution = solution[:]
    n = len(new_solution)
    if flag == 'intra':
        node1, node2 = swap
        node1_idx, node2_idx = new_solution.index(node1), new_solution.index(node2)
        if node1_idx > node2_idx:
            node1, node2 = node2, node1
            node1_idx, node2_idx = node2_idx, node1_idx
        prev_node1, next_node1 = new_solution[node1_idx-1], new_solution[(node1_idx+1)%n]
        prev_node2, next_node2 = new_solution[node2_idx-1], new_solution[(node2_idx+1)%n]
        if (node1_idx+1)%n == node2_idx or (node2_idx+1)%n == node1_idx:
            if node1_idx==0 and node2_idx==len(new_solution)-1:
                prev_node1, node1, node2, next_node2 = prev_node2, node2, node1, next_node1
            elif node1_idx > node2_idx:
                node1_idx, node2_idx = node2_idx, node1_idx
                prev_node1, next_node2 = prev_node2, next_node1
                node1, node2 = node2, node1
            old_cost = distance_matrix[prev_node1][node1] + distance_matrix[node1][node2] + distance_matrix[node2][next_node2]
            new_cost = distance_matrix[prev_node1][node2] + distance_matrix[node2][node1] + distance_matrix[node1][next_node2]
        else:
            old_cost = distance_matrix[prev_node1][node1] + distance_matrix[node1][next_node1] + \
                       distance_matrix[prev_node2][node2] + distance_matrix[node2][next_node2]
            new_cost = distance_matrix[prev_node1][node2] + distance_matrix[node2][next_node1] + \
                       distance_matrix[prev_node2][node1] + distance_matrix[node1][next_node2]
        delta = old_cost - new_cost

        new_solution[node1_idx], new_solution[node2_idx] = new_solution[node2_idx], new_solution[node1_idx]
        
    elif flag == 'inter':
        solution_node, outer_node = swap
        solution_node_idx = new_solution.index(solution_node)
        prev_solution_node, next_solution_node = new_solution[solution_node_idx-1], new_solution[(solution_node_idx+1)%n]

        old_cost = distance_matrix[prev_solution_node][solution_node] + distance_matrix[solution_node][next_solution_node] + \
                   costs[solution_node]
        new_cost = distance_matrix[prev_solution_node][outer_node] + distance_matrix[outer_node][next_solution_node] + \
                   costs[outer_node]
        
        delta = old_cost - new_cost
        new_solution[solution_node_idx] = outer_node

    return delta, new_solution

In [ ]:
def compute_edge_swap_delta(solution, swap, distance_matrix):
    (edge1_start, edge1_end), (edge2_start, edge2_end) = swap
    idx_edge1_start = solution.index(edge1_start)
    idx_edge2_start = solution.index(edge2_start)
    
    old_distance = distance_matrix[edge1_start][edge1_end] + distance_matrix[edge2_start][edge2_end]
    new_distance = distance_matrix[edge1_start][edge2_start] + distance_matrix[edge1_end][edge2_end]
    
    delta = old_distance - new_distance


    if idx_edge1_start < idx_edge2_start:
        new_solution = solution[:idx_edge1_start+1] + solution[idx_edge1_start+1:idx_edge2_start+1][::-1] + \
                       solution[idx_edge2_start+1:] 
    else:
        new_solution = solution[:idx_edge2_start+1] + solution[idx_edge2_start+1:idx_edge1_start+1][::-1] + \
                       solution[idx_edge1_start+1:]
    return delta, new_solution

In [ ]:
def steepest_local_search(solution, distance_matrix, costs, flag):
    
    if flag == 'edges':
        while True:
            neighborhood_edges = get_edge_moves(solution)
            neighborhood_edges = list(zip( neighborhood_edges, ['edge']*len(neighborhood_edges) ))
            
            neighborhood_nodes = get_node_moves(solution, 'inter')
            neighborhood_nodes = list(zip( neighborhood_nodes, ['node']*len(neighborhood_nodes) ))
            
            neighborhood = neighborhood_edges + neighborhood_nodes
            
            best_delta, best_solution = 0, None
            for i, neighbor in enumerate(neighborhood):
                if neighbor[1] == 'edge':
                    delta, new_solution = compute_edge_swap_delta(solution, neighbor[0], distance_matrix)
                elif neighbor[1] == 'node':
                    delta, new_solution  = compute_node_swap_delta(solution, neighbor[0], distance_matrix, costs, 'inter')
                if delta > best_delta:
                    best_delta = delta
                    best_solution = new_solution[:]
            
            if best_solution is not None:
                solution = best_solution[:]
                continue
            return solution
        
    elif flag == 'nodes':
        while True:
            neighborhood_intra = get_node_moves(solution, 'intra')
            neighborhood_intra = list(zip( neighborhood_intra, ['intra']*len(neighborhood_intra) ))
            
            neighborhood_inter = get_node_moves(solution, 'inter')
            neighborhood_inter = list(zip( neighborhood_inter, ['inter']*len(neighborhood_inter) ))
            
            neighborhood = neighborhood_inter + neighborhood_intra
            best_delta, best_solution = 0, None
            for i, neighbor in enumerate(neighborhood):
                delta, new_solution = compute_node_swap_delta(solution, neighbor[0], distance_matrix, costs, neighbor[1])
                if delta > best_delta:
                    best_delta = delta
                    best_solution = new_solution[:]
            
            if best_solution is not None:
                solution = best_solution[:]
                continue
            return solution
            

In [ ]:
columns = ["Algorithm", "TSPA", "TSPB", "TSPC", "TSPD"]
cost_df = pd.DataFrame(columns=columns)
time_df = pd.DataFrame(columns=columns)
best_solutions = {}

search_types = {'Steepest': steepest_local_search}
starting_solutions = {'Random': random_search}
instances = ['TSPA', 'TSPB', 'TSPC', 'TSPD']
path = "../data/"
for search_type in search_types:
    for flag in ('edges'):
        for starting_solution in starting_solutions:
            algo_name = f'{search_type}-{flag}-{starting_solution}'
            new_row = pd.DataFrame({columns[0]: algo_name}, index=[0])
            cost_df = pd.concat([cost_df, new_row], ignore_index=True)
            time_df = pd.concat([time_df, new_row], ignore_index=True)
            best_solutions_tmp = {}
            for instance in instances:
                file_name = f'{path}{instance}.csv'
                df = pd.read_csv(file_name, names=['x', 'y', 'cost'], sep=';')
                distance_matrix = get_distance_matrix(df)
                distance_matrix_with_costs = get_distance_matrix(df, True)
                costs = df.cost.values
                total_costs, solutions, times = list(), list(), list()
                print(algo_name, instance)
                for i in range(200):
                    print(i)
                    solution, _ = starting_solutions[starting_solution](distance_matrix_with_costs)
                    start_time = perf_counter()
                    solution = search_types[search_type](solution, distance_matrix, costs, flag)
                    total_cost = sum(distance_matrix_with_costs[solution[i]][solution[i+1]] for i in range(len(solution) - 1))
                    total_cost += distance_matrix_with_costs[solution[-1]][solution[0]]
                    end_time = perf_counter()
                    total_costs.append(total_cost)
                    solutions.append(solution)
                    times.append(round(end_time - start_time, 3))
                best_solution_idx = np.argmin(total_costs)
                best_solutions_tmp[file_name] = solutions[best_solution_idx]
                cost_df.at[cost_df.index[-1], instance] = f'{np.mean(total_costs)} ({np.min(total_costs)} - {np.max(total_costs)})'
                time_df.at[time_df.index[-1], instance] = f'{round(np.mean(times), 3)} ({np.min(times)} - {np.max(times)})'
                print(f'{np.mean(total_costs)} ({np.min(total_costs)} - {np.max(total_costs)})')
                print(f'{np.mean(times)} ({np.min(times)} - {np.max(times)})')
                print(solutions[best_solution_idx])
                display(cost_df)
                display(time_df)
            best_solutions[algo_name] = best_solutions_tmp

In [ ]:
cost_df.to_csv('costs.csv')

In [ ]:
time_df.to_csv('times.csv')

In [ ]:
display(cost_df)

In [ ]:
display(time_df)

In [ ]:
for algo in best_solutions:
    fig, ax = plt.subplots(2, 2, figsize=(25, 25))
    ax = ax.flatten()
    idx = 0
    fig.suptitle(algo, fontsize=40)
    for instance in best_solutions[algo]:
        solution = best_solutions[algo][instance]
        df = pd.read_csv(instance, names=['x', 'y', 'cost'], sep=';')
        weights = df['cost']
        cmap = plt.cm.viridis
        norm = mcolors.Normalize(vmin=min(weights), vmax=max(weights))
        ax[idx].set_title(instance.replace('../data/', '').replace('.csv', ''), fontsize=30)
        for i in range(len(df)):
            x, y, cost = df.iloc[i]['x'], df.iloc[i]['y'], df.iloc[i]['cost']
            ax[idx].plot(x, y, "o", markersize=20, color=cmap(norm(cost)))
        for i in range(len(solution)-1):
            x, y, cost = df.iloc[solution[i]]['x'], df.iloc[solution[i]]['y'], df.iloc[solution[i]]['cost']
            x_next, y_next = df.iloc[solution[i+1]]['x'], df.iloc[solution[i+1]]['y']
            ax[idx].plot((x, x_next), (y, y_next), "-", color='black')
        x, y = df.iloc[solution[0]]['x'], df.iloc[solution[0]]['y']
        ax[idx].plot((x, x_next), (y, y_next), "-", color='black')
        
        axins = ax[idx].inset_axes([1.05, 0.1, 0.05, 0.6], transform=ax[idx].transAxes)
        gradient = np.linspace(0, 1, 256).reshape(-1, 1)

        axins.imshow(gradient, aspect='auto', cmap=cmap, origin='lower', extent=[0, 1, min(weights), max(weights)])
        axins.xaxis.set_visible(False)
        
        idx += 1
        
    plt.savefig(f'./plots/{algo}.png', dpi=300)
    plt.show()
    